In [33]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.carvana.com/cars")
time.sleep(20)


company_list = ['Acura', 'Alfa Romeo', 'Audi', 'BMW', 'Buick', 'Cadillac', 'Chevrolet', 'Chrysler', 'Dodge', 'FIAT', 'Ford', 'Genesis', 'GMC', 'Honda', 'Hyundai', 'INFINITI', 'Jaguar', 'Jeep', 'Kia', 'Land Rover', 'Lexus', 'Lincoln', 'Lucid', 'Maserati', 'Mazda', 'Mercedes-Benz', 'MINI', 'Mitsubishi', 'Nissan', 'Polestar', 'Porsche', 'Ram', 'Rivian', 'Scion', 'smart', 'Subaru', 'Tesla', 'Toyota', 'Volkswagen', 'Volvo']

location = driver.find_element(By.XPATH, '//*[@id="search-tools"]/div[1]/div/span').text
element = driver.find_elements(By.XPATH, "//a[@href]")

links = []
pattern = re.compile(r'https://www.carvana.com/vehicle/\d+')

for i in element:
    links.append(i.get_attribute("href"))
filtered_list = [item for item in links if pattern.match(item)]
concatenated_list = [item + "#vehicle-details" for item in filtered_list]

'''
base_url = 'https://www.carvana.com/cars?page='
# Considering 2 to 5 pages only
page_list = set([base_url + str(page_number) for page_number in range(2, 6)])

for i in page_list:
    print(page_list)
'''

"\nbase_url = 'https://www.carvana.com/cars?page='\n# Considering 2 to 5 pages only\npage_list = set([base_url + str(page_number) for page_number in range(2, 6)])\n\nfor i in page_list:\n    print(page_list)\n"

In [34]:
sample_list = [concatenated_list[0], concatenated_list[1]]
data_to_append = []
n = 1

for url in concatenated_list:
    driver.get(url)
    time.sleep(10)

    element = driver.find_elements(By.ID, '__next')
    words = element[0].text.split('\n')
    car = ""
    car_name = ""
    company_name = ""
    model = ""
    year = ""
    price = ""
    miles = ""
    car_category = ""
    fuel_type = ""
    auto_man = ""
    auto_man_1 = ""
    auto_man_2 = ""
    VIN = ""
    for i in range(0,len(words)):
        if(re.match(r'\d{1,3}(?:,\d{3})*(?:\.\d+)? miles', words[i])):
            miles = words[i]
        if not price:
            if(re.match(r'\$\d{1,3}(?:,\d{3}){0,2}', words[i])):
                price = words[i]
        if(re.match(r'\s*20\d{2}', words[i])):
            car = words[i]
            car_name = ' '.join(car.split()[1:])
            for company in company_list:
                if(company in car_name):
                    company_name = company
                    model = car_name.replace(company_name, '')
                else:
                    model = car_name
            year = car.split(" ")[0]
            car_category = words[i+1]
        if(words[i].startswith("VIN")):
            VIN = words[i]

        # To do : Improve logic for Fuel and Auto/Manual
        if(words[i]=="Fuel"):
            fuel_type = words[i+1]
        if(words[i]=="Transmission"):
            if(words[i+1].split(" ")>1):
                auto_man_1 = words[i+1]
        if((("Auto" in words[i]) or ("Manual" in words[i])) and ("This") not in words[i]):
            auto_man_2 = words[i]
    
    if not fuel_type:
        for i in range(0,len(words)):
            if "Electric Motor" in words[i]:
                fuel_type = "Electric"
            if ((re.match(r'^\d-Cyl', words[i])) or ((words[i].startswith("V6") or (words[i].startswith("V8")) and words[i].endswith("Liter")))):
                fuel_type ="Gas"
    
    if fuel_type == "Electric":
        auto_man = "Automatic as it's an Electric Car"
    elif auto_man_1:
        auto_man = auto_man_1
    else:
        auto_man = auto_man_2

    data_to_append.append((n, url.replace("#vehicle-details", ''), company_name, model, price, (VIN.split(" ")[1]),  
                            year, miles, car_category, fuel_type, (auto_man.split(',')[0]), location))
    n = n+1
driver.quit()

In [35]:
df = pd.DataFrame(data_to_append, columns=['Sr. no.', 'Link', 'Company', 'Model', 'Price', 'VIN', 'Year', 'Miles', 
                                            'Category', 'Fuel', 'Auto/Manual', 'Location'])
display(df)

,Sr. no.,Link,Company,Model,Price,VIN,Year,Miles,Category,Fuel,Auto/Manual,Location
0,1,https://www.carvana.com/vehicle/3012206,BMW,BMW M3,"$71,990",WBS33AY05NFL88600,2022,"27,722 miles",Competition Sedan 4D,Gas,Automatic,"Atlanta, GA"
1,2,https://www.carvana.com/vehicle/3011873,Lexus,Lexus RX,"$42,990",2T2SZMDA0MC303909,2021,"31,315 miles",RX 350 F SPORT SUV 4D,Gas,Automatic,"Atlanta, GA"
2,3,https://www.carvana.com/vehicle/3019352,Land Rover,Land Rover Discovery Sport,"$22,990",SALCP2FX2KH802071,2019,"47,726 miles",SE Sport Utility 4D,Gas,Automatic,"Atlanta, GA"
3,4,https://www.carvana.com/vehicle/3002335,Lexus,Lexus UX,"$27,990",JTHX3JBH8M2036171,2021,"32,991 miles",200 Sport Utility 4D,Gas,Automatic,"Atlanta, GA"
4,5,https://www.carvana.com/vehicle/3012169,Mercedes-Benz,Mercedes-Benz Mercedes-AMG CLA,"$30,990",WDDSJ5CB6GN392646,2016,"45,272 miles",CLA 45 4MATIC Coupe 4D,Gas,Automatic,"Atlanta, GA"
5,6,https://www.carvana.com/vehicle/3002211,Ford,Ford Escape,"$13,590",1FMCU0D72BKB58660,2011,"51,599 miles",XLT Sport Utility 4D,Gas,Automatic,"Atlanta, GA"
6,7,https://www.carvana.com/vehicle/3017314,Audi,Audi A4,"$13,990",WAULF78K29N075287,2009,"40,448 miles",2.0T Quattro Sedan 4D,Gas,Auto,"Atlanta, GA"
7,8,https://www.carvana.com/vehicle/3010272,Land Rover,Land Rover Discovery Sport,"$19,590",SALCR2BG6GH619845,2016,"60,303 miles",HSE Sport Utility 4D,Gas,Automatic,"Atlanta, GA"
8,9,https://www.carvana.com/vehicle/2968602,Volvo,XC40 Recharge,"$30,990",YV4ED3UR3M2452052,2021,"25,660 miles",Sport Utility 4D,Electric,Automatic as it's an Electric Car,"Atlanta, GA"
9,10,https://www.carvana.com/vehicle/3013910,Maserati,Maserati Ghibli,"$25,590",ZAM57XSA3G1187167,2016,"41,313 miles",Sedan 4D,Gas,Auto,"Atlanta, GA"


In [36]:
df.to_excel("output_new.xlsx")  